In [1]:
from math import sqrt
import pandas as pd

df = pd.read_csv('telemetry-rio-5-laps.csv', decimal='.').loc[:, ['position_x', 'position_y', 'position_z', 'lap_number', 'gear', 'speed', 'current_engine_rpm', 'fuel', 'wheel_on_rumble_strip_front_left', 'wheel_on_rumble_strip_front_right', 'wheel_on_rumble_strip_rear_left', 'wheel_on_rumble_strip_rear_right']]

first_x, first_y = 809.8262, 141.70494
second_x, second_y = -694.0124, 299.16605
thirdy_x, thirdy_y = 342.6821, 289.19034


radius = 125
first = True
second = False
thirdy = False
i = 0

df['sector'] = None

for index, row in df.iterrows():
    if not pd.isnull(row['sector']):
        continue
    
    if first:
        if pow(row["position_x"] - second_x, 2) + pow(row["position_z"] - second_y, 2) < radius:
            first = False
            second = True
        
        df.at[index, "sector"] = "sector1"
        continue
            
    if second:
        if pow(row["position_x"] - thirdy_x, 2) + pow(row["position_z"] - thirdy_y, 2) < radius:
            second = False
            thirdy = True
        
        df.at[index, "sector"] = "sector2"
        continue
    
    if thirdy:
        if pow(row["position_x"] - first_x, 2) + pow(row["position_z"] - first_y, 2) < radius:
            thirdy = False
            first = True
        
        df.at[index, "sector"] = "sector3"
        continue

df.to_csv('CircuitData.csv')